In [5]:
# load generated questions
import numpy as np
from collections import Counter, defaultdict

topics = defaultdict(list)
qas = defaultdict(list)
non_unique_questions = defaultdict(list)
with open('../results/cast19_test/t5_cast19_test_paragraphs_17.tsv', 'r') as fin:
    for line in fin:
        cells = line.rstrip('\n').split('\t')
        q_id = cells[0]
        p_id = cells[1]
        topic = q_id.split('_')[0]
        
        # skip duplicate questions
        qs = set(cells[2:])
    
        for q in qs:
            # store all questions per topic
            if q not in topics[topic]:
                topics[topic].append(q)
            # store all passages per question
            if p_id not in qas[q]:
                qas[q].append(p_id)
            # store all topics per question
            if topic not in non_unique_questions[q]:
                non_unique_questions[q].append(topic)

In [10]:
# same as in TREC
min_passages_per_question = 3

doc_id = 0
documents = []
for topic, qs in topics.items():
#     print(topic)
    for q in qs:
        # index questions with sufficient support (non-unique within topic)
        if len(qas[q]) < min_passages_per_question:
            continue
        # index questions unique across topic
        if len(non_unique_questions[q]) > 1:
            continue
#         print(q)
#         print(qas[q])
        documents.append({'id': doc_id,
                          'contents': q})
        doc_id += 1
#     break
print(len(documents), 'questions')

2688 questions


In [ ]:
# Create json document files.
anserini_folder = 
json_files_path = "../results/questions_index/collection"
path_index = "../results/questions_index/index"

if not os.path.isdir(json_files_path):
    os.makedirs(json_files_path)
    for i, doc in enumerate(documents):
        with open(json_files_path+'/docs{:02d}.json'.format(i), 'w', encoding='utf-8', ) as f:
            f.write(json.dumps(doc) + '\n')        

    # Run index java command
    os.system("sh {}target/appassembler/bin/IndexCollection -collection JsonCollection"   \
                " -generator DefaultLuceneDocumentGenerator -threads 9 -input {}" \
                " -index {}anserini_index -storePositions -storeDocvectors -storeRaw". \
                format(anserini_folder, json_files_path, path_index))

In [17]:
import random

# same as in TREC
min_passages_per_question = 3

for topic, qs in topics.items():
    print(topic)
    
    # sample the first question at random
    while True:
        q1 = random.sample(qs, 1)[0]
        if len(qas[q1]) >= min_passages_per_question:
            print(q1)
            break
            
    # sample next questions by overlap
    break

31
where does throat cancer occur


In [8]:
# build fingerprints
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
  
lemmatizer = WordNetLemmatizer() 
tokenizer = RegexpTokenizer(r'\w+')
stopwords = stopwords.words('english')


# same as in TREC
min_passages_per_question = 3

t_qs = defaultdict(list)
t_as = defaultdict(list)
topic_words = {}

for topic, qs in topics.items():
    print(topic)
    words = Counter()
    for q in qs:
        if len(qas[q]) < min_passages_per_question:
            continue
        
        question_words = [lemmatizer.lemmatize(w) for w in tokenizer.tokenize(q.lower()) if w not in stopwords]
        words.update(question_words)
        print(question_words)
        t_qs[topic].append(q)
        t_as[topic].extend(qas[q])
    
    frequent_words = [w for w in words if words[w] > 1]
    print(frequent_words)
    topic_words[topic] = frequent_words
        
    break

31
['cause', 'throat', 'cancer']
['throat', 'cancer']
['cause', 'throat', 'cancer']
['squamous', 'cell', 'carcinoma']
['common', 'throat', 'cancer']
['symptom', 'throat', 'cancer']
['cure', 'throat', 'cancer']
['cause', 'throat', 'cancer']
['treatment', 'throat', 'cancer']
['host', 'loose', 'end']
['treatment', 'throat', 'cancer']
['throat', 'cancer', 'called']
['common', 'cause', 'lung', 'cancer']
['type', 'cancer', 'throat']
['cause', 'throat', 'cancer']
['cause', 'throat', 'cancer']
['throat', 'cancer']
['throat', 'cancer', 'caused']
['cause', 'cancer', 'throat']
['nasopharyngeal', 'cancer']
['different', 'type', 'throat', 'cancer']
['throat', 'cancer']
['pharyngeal', 'cancer']
['cause', 'throat', 'cancer', 'woman']
['cause', 'cancer']
['cause', 'throat', 'cancer']
['cause', 'throat', 'cancer']
['pharyngeal', 'cancer']
['throat', 'cancer']
['type', 'throat', 'cancer']
['different', 'type', 'throat', 'cancer']
['type', 'cancer', 'throat']
['neck', 'cancer', 'symptom']
['cause', 'neck

['esophageal', 'cancer', 'symptom']
['esophagus', 'cancer']
['esophageal', 'cancer']
['esophageal', 'cancer']
['esophagus', 'cancer', 'symptom']
['esophagus', 'cancer']
['esophagus', 'cancer']
['type', 'cancer', 'esophageal']
['esophagus', 'cancer']
['type', 'esophageal', 'cancer']
['cancer', 'esophagus']
['symptom', 'throat', 'cancer']
['symptom', 'throat', 'cancer']
['sign', 'throat', 'cancer']
['symptom', 'throat', 'cancer', 'child']
['symptom', 'throat', 'cancer']
['symptom', 'throat', 'cancer']
['symptom', 'throat', 'cancer']
['cancer', 'throat', 'symptom']
['esophagus']
['esophageal', 'cancer', 'symptom']
['esophageal', 'cancer']
['symptom', 'esophageal', 'cancer']
['esophageal', 'cancer', 'symptom']
['esophagus', 'cancer']
['symptom', 'esophageal', 'cancer']
['esophageal']
['throat', 'cancer']
['type', 'cancer', 'throat']
['throat', 'cancer']
['cause', 'difficulty', 'swallowing']
['esophageal', 'cancer']
['esophagus', 'cancer']
['type', 'cancer', 'esophageal']
['esophagus', 'can

In [20]:
# encode questions into fingerprints
import numpy as np

for topic, frequent_words in topic_words.items():
    question_sequence = []
    qs = t_qs[topic]
    print('dialogue vocabulary', len(frequent_words))
    print(frequent_words)
    fp = np.zeros((len(qs), len(frequent_words)))
    for i, q in enumerate(qs):
        for w in [lemmatizer.lemmatize(w) for w in tokenizer.tokenize(q.lower())]:
            if w in frequent_words:
                fp[i][frequent_words.index(w)] = 1
    print(fp)
    print('\n')
    # show word roles
#     for j, w in enumerate(frequent_words):
#         print(w, fp[:, j])


    # 0 sample the first question at random
    q1 = random.sample(range(len(qs)), 1)[0]
    print(qs[q1])
    print(fp[q1])
    
    # 1+ select related questions by overlap
            
    break

dialogue vocabulary 95
['cause', 'throat', 'cancer', 'squamous', 'cell', 'carcinoma', 'common', 'symptom', 'cure', 'treatment', 'host', 'loose', 'end', 'called', 'lung', 'type', 'caused', 'nasopharyngeal', 'different', 'pharyngeal', 'woman', 'neck', 'head', 'located', 'laryngeal', 'part', 'hpv', 'kind', 'oropharyngeal', 'sign', 'esophageal', 'esophagus', 'oral', 'mouth', 'best', 'occur', 'mean', 'spread', 'tobacco', 'pilocarpine', 'used', 'tablet', 'cured', 'curable', 'treat', 'adult', 'anemia', 'small', 'classification', 'main', 'usually', 'two', 'non', 'smoking', 'risk', 'stage', 'definition', 'medical', 'cigar', 'leading', 'usa', 'u', 'chest', 'pain', 'nonsmall', 'begin', 'metastatic', 'originate', 'start', 'pulmonary', 'shortness', 'breath', 'early', 'colon', 'liver', 'human', 'cough', 'diagnosed', 'sclc', 'brain', 'cigarette', 'sore', 'related', 'lining', 'coughing', 'lump', 'difficulty', 'swallowing', 'first', 'know', 'md', 'come', 'found', 'child', 'oesophageal']
[[1. 1. 1. ... 